In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import vmap
import pandas as pd
import tensorcircuit as tc
from tensorcircuit import shadows
from functools import partial
import pennylane as qml
from tqdm import tqdm
import basefunction



# 

In [ ]:

state_vectors = {
    0: np.array([1, 0]),
    1: np.array([0, 1]),
    2: np.array([1, 1]) / np.sqrt(2),
    3: np.array([1, -1]) / np.sqrt(2),
    4: np.array([1, 1j]) / np.sqrt(2),
    5: np.array([1, -1j]) / np.sqrt(2),
}

def large_scale_circuit(params, num_qubits, initial_state):
    tc.set_backend("jax")
    c = tc.Circuit(num_qubits, inputs=initial_state)
    n_layers = params.shape[0]
    for layer in range(n_layers):
        for qubit in range(num_qubits):
            c.rx(qubit, theta=params[layer, qubit, 0])
            c.ry(qubit, theta=params[layer, qubit, 1])
            c.rz(qubit, theta=params[layer, qubit, 2])
        for qubit in range(num_qubits - 1):
            c.cnot(qubit, qubit + 1)
    return c.state()

def test_circuit(inistr, params):
    num_qubits = len(inistr)
    initial_state = basefunction.str2rhoinistr(inistr)
    state_ode = large_scale_circuit(params, num_qubits, initial_state)
    return state_ode

def read_data():
    try:
        df = pd.read_csv('qubit_data.csv')
        result_dict = {}
        for _, row in df.iterrows():
            num_qubits = row['num_qubits']
            rhoinput = np.array([int(x) for x in row['rhoinput'].split(',')])
            params_flat = np.array([float(x) for x in row['params'].split(',')])
            params = params_flat.reshape(10, num_qubits, 3)
            result_dict[num_qubits] = {'rhoinput': rhoinput, 'params': params}
        return result_dict
    except FileNotFoundError:
        print("Error: File not found!")
        return None

def shadow_creator(num_qubits, params, nshot):
    resultin = basefunction.generate_random_sublists(nshot, num_qubits)
    state_p = jax.vmap(test_circuit, in_axes=(0, None))(resultin, params)
    tc.set_backend("jax")
    pauli_strings = tc.backend.convert_to_tensor(np.random.randint(1, 4, size=(nshot, num_qubits)))
    status = tc.backend.convert_to_tensor(np.random.rand(nshot, 1))
    @partial(tc.backend.jit, static_argnums=(3,))
    def shadow_ss(psi, pauli_strings, status, measurement_only=True):
        return shadows.shadow_snapshots(
            psi, jnp.array([pauli_strings]), jnp.array([status]), measurement_only=measurement_only
        )
    vmap_shadow_ss = jax.vmap(shadow_ss, in_axes=(0, 0, 0), out_axes=0)
    ss_states = vmap_shadow_ss(jnp.array(state_p), jnp.array(pauli_strings), jnp.array(status))
    resultout = jax.vmap(basefunction.shadow2index2, in_axes=0)(ss_states, pauli_strings - 1)
    return np.array(resultin), np.array(resultout)

if __name__ == "__main__":
    data = read_data()
    for num_qubits in [2, 4, 6, 8]:
        jax.config.update("jax_enable_x64", True)
        rhoinput = data[num_qubits]["rhoinput"]
        params = data[num_qubits]["params"]
        N = 100000
        num_runs = 50
        all_runs = []
        for run_id in tqdm(range(num_runs), desc="Processing runs"):
            resultin, resultout = shadow_creator(num_qubits, params, N)
            combined = np.column_stack((
                np.full(N, run_id, dtype=int),
                resultin,
                resultout
            ))
            all_runs.append(combined)
        final_data = np.vstack(all_runs)
        np.savetxt(f'{num_qubits}qubits_performace_shadows.csv', final_data,
                   delimiter=',',
                   header='exp_id,resultin,resultout',
                   fmt='%d',
                   comments='')
    